In [2]:
# System
import pandas as pd
import numpy as np
import time
from selenium import webdriver

In [3]:
#Params

driver_path = '../chromedriver_win32/chromedriver.exe'

url = 'https://www.catho.com.br/vagas'

#job_to_search = input('Escreva a vaga procurada.')
#city_or_region = input('Escreva a cidade, estado ou região de interesse.')

job_to_search = 'Cientista de Dados'

city_or_region = 'São Paulo'

#wage = '8.000'


In [4]:
# open the chrome browser

driver = webdriver.Chrome(executable_path=driver_path)

driver.get(url)

In [5]:
# Setting search

# selecting job_name field and clear remaining entries:
job = driver.find_element_by_xpath('/html/body/div[1]/article/div[1]/section/article/form/label/input')
job.clear()

# writing on the element
job.send_keys(job_to_search)

# selecting city_State_or_Region field:
region = driver.find_element_by_xpath('/html/body/div[1]/article/div[1]/section/article/form/div[1]/input[5]')
region.clear()

# writing on the element
region.send_keys(city_or_region)

time.sleep(1)

# Selecting the Search Button
search_bttn = driver.find_element_by_xpath('/html/body/div[1]/article/div[1]/section/article/form/div[2]/input')
search_bttn.click()

In [6]:
#web-scraping
# Extraction

def extractor(seq_num):
    """
    Finds the job description box of number seq_num, extracts it's information and returns pandas DataFrame

    """
    #target = driver.find_element_by_xpath(f'/html/body/div[1]/article/section/ul/li[{seq_num}]/section')
    
    published = driver.find_element_by_xpath(f'/html/body/div[1]/article/section/ul/li[{seq_num}]/section/div/header/h2/time')
    name = driver.find_element_by_xpath(f'/html/body/div[1]/article/section/ul/li[{seq_num}]/section/div/header/h2/a')
    salary = driver.find_element_by_xpath(f'/html/body/div[1]/article/section/ul/li[{seq_num}]/section/div/header/h2/div/strong[1]')
    n_jobs = driver.find_element_by_xpath(f'/html/body/div[1]/article/section/ul/li[{seq_num}]/section/div/header/h2/div/strong[2]/span[1]')
    city = driver.find_element_by_xpath(f'/html/body/div[1]/article/section/ul/li[{seq_num}]/section/div/header/h2/div/strong[2]/span[2]')  
    description = driver.find_element_by_xpath(f'/html/body/div[1]/article/section/ul/li[{seq_num}]/section/div/article/div[1]/div')
    
    return pd.DataFrame({'job_name':[name.text], 'publicated_in':[published.text], 'salary':[salary.text], 'n_jobs':[n_jobs.text], 'city':[city.text], 'description':[description.text]})

#_________________________________________________________________________________

def move_foward():
    """
    Moves to the next page, if there is any, and returns True. Else, return False.
    """
    try:
        a = driver.find_element_by_tag_name('a.arrow.next')
        a.click()
        return True
    except:
        return False
#___________________________________________________________________________________

def page_scanner():
    """
    something
    """
    df1 = extractor(1)

    for i in range(2,11):
        try:
            df1 = pd.concat([df1,extractor(i)]) 
        except:
            pass
    
    return df1

In [7]:
df = page_scanner()

condition = move_foward()

while condition == True:
    try:
        df1 = page_scanner()
        df = pd.concat([df, df1])
        time.sleep(1)
        condition = move_foward()
    except:
        pass
    

In [8]:
df = df.reset_index(drop=True)

In [9]:
df

,job_name,publicated_in,salary,n_jobs,city,description
0,Cientista de Dados,"quinta, 16/04",A Combinar,1 vaga:,São Paulo-SP(1),Linguagens Python ou R - Estatística básica: c...
1,Cientista de Dados,"segunda, 13/04",A Combinar,1 vaga:,São Paulo-SP(1),Description: Our Client focuses on helping cli...
2,Cientista de Dados,"quinta, 19/03",A Combinar,1 vaga:,São Paulo-SP(1),"Somos um time ágil, autônomo, multidisciplinar..."
3,Cientista de Dados,"terça, 03/03",A Combinar,1 vaga:,São Paulo-SP(1),Atuar em empresa multinacional focada em desen...
4,Cientista de Dados,"sexta, 20/12",A Combinar,1 vaga:,São Paulo-SP(1),"Produzir análises detalhadas, por meio da inte..."
...,...,...,...,...,...,...
75,Coordenador de Business Intelligence,"quarta, 15/01",A Combinar,1 vaga:,Ribeirão Preto-SP(1),Profissional será responsável pela Área de B.I...
76,Cientista de Dados,"quarta, 01/04",A Combinar,1 vaga:,Vinhedo-SP(1),"Precisamos de você, Cientista de Dados que tem..."
77,Cientista de Dados,"quinta, 12/03",A Combinar,1 vaga:,Ribeirão Preto-SP(1),Participar ativamente de diversas iniciativas ...
78,Cientista de Dados,"segunda, 02/03",A Combinar,1 vaga:,Campinas-SP(1),"Elaboração, implantação e manutenção de modelo..."


In [10]:
df.shape

(80, 6)

In [11]:
df['python'] = df['description'].apply(lambda x: 1 if 'python' in x.lower() else 0)

In [12]:
df[df['python'] == 1]

,job_name,publicated_in,salary,n_jobs,city,description,python
0,Cientista de Dados,"quinta, 16/04",A Combinar,1 vaga:,São Paulo-SP(1),Linguagens Python ou R - Estatística básica: c...,1
10,Cientista de Dados,"quinta, 16/04",A Combinar,1 vaga:,São Paulo-SP(1),Linguagens Python ou R - Estatística básica: c...,1
17,Cientista de Dados / Spark / Python Sênior,"sexta, 06/03",A Combinar,1 vaga:,São Paulo-SP(1),Data Science utilizando Spark e Python. Modelo...,1


In [13]:
df['salary'].unique()

array(['A Combinar', 'R$ 7.000,00', 'R$ 3.500,00',
       'De R$ 5.001,00 a R$ 6.000,00', 'R$ 4.000,01',
       'De R$ 4.001,00 a R$ 5.000,00', 'De R$ 2.001,00 a R$ 3.000,00',
       'De R$ 10.001,00 a R$ 15.000,00', 'De R$ 6.001,00 a R$ 7.000,00',
       'De R$ 8.001,00 a R$ 9.000,00'], dtype=object)

In [14]:
df[df['salary']=='De R$ 10.001,00 a R$ 15.000,00']

,job_name,publicated_in,salary,n_jobs,city,description,python
51,Coordenador de Business Intelligence,"sexta, 13/03","De R$ 10.001,00 a R$ 15.000,00",1 vaga:,São Paulo-SP(1),"Captação, gestão e tratamento de dados relacio...",0
